### Imports

In [ ]:
import numpy as np
import pandas as pd
# import xarray as xr
import geopandas
import time, pickle
import torch
from torch.utils.data import DataLoader

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from models import *
from utils import *
from runmanager import *
from experiment import *
from plot_utils import *
from preprocessing_utils import *

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

pd.options.display.max_columns = None
np.random.seed(4)

%load_ext autoreload
%autoreload 2

import matplotlib
matplotlib.rc_file_defaults()

In [ ]:
from elbo import ApproximatePosterior
from gp_mlp import forward_backward_pass, MapDataset, MultipleOptimizer, UpperIndusGridDataset, UpperIndusDataset, prepare_data

In [ ]:
import torch.nn as nn
from stheno.torch import B, GP, EQ, Normal, Measure, Matern12, Matern32, Matern52

# Detect device.
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
# Let all of Stheno run on that device.
B.on_device(device).__enter__()
    
# B.epsilon is the default value for the diagonal jitter of the matrix
B.epsilon = 1e-4  # Needs to be relatively high for `float32`s

In [ ]:
# Dates
start="1900-01-01"
end="2020-12-31"

In [ ]:
TRAIN_PATH = '../../data/idaweb/observations_enriched_WRF_reduced.pkl'
TEST_PATH = r'/users/marron31/data/idaweb/maps_for_predictions/1991-06-03.csv'

In [ ]:
predictant = ['Prec']
predictors = [
              #'Date',
              #'Station',
              #'Prec',
              #'Corrected Station Name', 
              'X', 'Y',
              'Z',
              #'Altitude (m)', 
              #'Z', 
              #'RAIN', #'wrf_bc_prcp', 
              #'elev_hr', 
              #'aspect','slope', 
              #'doy', 
              'doy_sin', 'doy_cos', 
              'W500', 'RH2', 'U500', 'V10', 'RH500', 'RAIN', 'U10', 'V500', 'T2MIN',
              'T2', 'T2MAX'
              #'wrf_prcp_-1', 'wrf_prcp_-2','wrf_prcp_1', 'wrf_prcp_2', 
              #'Basin', 'lon', 'lat', 
#               'era5_u', 'era5_v',
              #'era5_u_-2', 'era5_u_-1', 'era5_u_1', 'era5_u_2', 
              #'era5_v_-2', 'era5_v_-1', 'era5_v_1', 'era5_v_2'
             ]

In [ ]:
# Initially used to create split but then manually setting stations because 
# there is some issue with stations missing after data preparation process
# pick 10% random stations for validation, 10% for testing and the rest for training

# stations = list(pd.read_pickle(TRAIN_PATH).Station.unique())
# random.shuffle(stations)
# val_stations = stations[:int(len(stations)*0.1)]
# test_stations = stations[int(len(stations)*0.1):int(len(stations)*0.2)]
# train_stations = stations[int(len(stations)*0.2):]

In [ ]:
train_stations = ['ABO', 'AHO', 'AIR', 'ALV', 'APT', 'ARO', 'AUB', 'AVA', 'AVB', 'BAC', 'BAL', 'BEA', 'BEC', 'BEGTH', 'BEGWA', 'BEH', 'BEHAK', 'BEKSE', 'BERIF', 'BES', 'BEV', 'BIN', 'BIV', 'BLA', 'BRA', 'BRW', 'BSP', 'BSW', 'CAV', 'CDF', 'CHA', 'CHB', 'CHD', 'CHM', 'CHW', 'CIM', 'COV', 'CTA', 'CTO', 'CUE', 'DAV', 'DIB', 'DIS', 'DLBAL', 'DLFEB', 'DLZUG', 'DMA', 'DOL', 'EIT', 'ENG', 'ERN', 'EVL', 'EVO', 'FIL', 'FIO', 'FIT', 'FLI', 'FRE', 'GEN', 'GOA', 'GRC', 'GRH', 'GSB', 'GSG', 'GST', 'GTT', 'GWA', 'HER', 'HIR', 'HOE', 'INF', 'IYDEO', 'IYDEU', 'IYMMR', 'IYMUW', 'IYPEN', 'IYPFE', 'IYPFI', 'IYPLI', 'IYPRU', 'IYREI', 'IYRID', 'IYROT', 'IYSMG', 'IYSMT', 'IYSUL', 'IYSUM', 'IYSVP', 'IYSWA', 'IYTER', 'IYTOB', 'IYULT', 'IYVAA', 'IYVAL', 'IYWEL', 'IYWOL', 'KLA', 'KRO', 'KSE', 'LBA', 'LEH', 'LEU', 'LOC', 'LOE', 'MAL', 'MAT', 'MAU', 'MLS', 'MOD', 'MSO', 'MST', 'MTE', 'MUE', 'MUS', 'MVE', 'NABCHA', 'NABDAV', 'NAP', 'NEB', 'OBI', 'OBW', 'PDM', 'PIG', 'PIL', 'PLF', 'PON', 'PSB', 'PUD', 'REC', 'ROB', 'ROE', 'ROG', 'RUM', 'SAB', 'SAE', 'SAF', 'SAM', 'SAN', 'SAP', 'SAS', 'SBA', 'SBE', 'SCU', 'SDO', 'SED', 'SEP', 'SGD', 'SIA', 'SIM', 'SLFAM2', 'SLFEM2', 'SLFFIS', 'SLFGL2', 'SLFMEI', 'SLFOBM', 'SLFSA3', 'SLFSC2', 'SLFSC3', 'SLFTU2', 'SLFUR2', 'SLFURS', 'SNE', 'SOG', 'SRL', 'STP', 'SVG', 'SWA', 'TIBED', 'TIBIA', 'TICOL', 'TIFUS', 'TIOLI', 'TST', 'UNS', 'URB', 'VDLSP', 'VDSEP', 'VEL', 'VIO', 'VRI', 'VSANZ', 'VSARO', 'VSBRI', 'VSCHY', 'VSDUR', 'VSEMO', 'VSFIN', 'VSGDX', 'VSJEI', 'VSMAT', 'VSSAB', 'VSSTA', 'VST', 'VSVER', 'WAW', 'WET', 'ZER', 'ZEV', 'ZNZ']
val_stations = ['ANT', 'BEHAB', 'BIO', 'BOS', 'BRL', 'BUF', 'ELO', 'FIE', 'GRY', 'GUE', 'IYHIN', 'IYTAU', 'KAS', 'LAT', 'LBG', 'MEB', 'RIE', 'SLFSH2', 'SLFSP2', 'SMM', 'SMZ', 'TICVM', 'VAB', 'VSMOI', 'VSTSN']
len(train_stations), len(val_stations)

### Dataloaders

In [ ]:
ds_dataset = UpperIndusDataset(TRAIN_PATH, start, end, predictant, predictors, stations=train_stations)
ds_dataset_val = UpperIndusDataset(TRAIN_PATH, start, end, predictant, predictors, stations=val_stations)

train_mean = ds_dataset.mean
train_var = ds_dataset.var

train_dataloader = DataLoader(dataset=ds_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=ds_dataset_val, batch_size=16, shuffle=True)

In [ ]:
# ds_dataset_test = UpperIndusDataset(TRAIN_PATH, start, end, predictant, predictors, stations=test_stations)
# ds_dataset_map = UpperIndusDataset(TEST_PATH, start, end, predictant, predictors, stations=test_stations)
# dataloader_test = DataLoader(dataset=ds_dataset, batch_size=16, shuffle=True)

# ds_dataset_map = MapDataset(TEST_PATH, predictors, train_mean, train_var)
# dataloader_map = DataLoader(dataset=ds_dataset_map, batch_size=1, shuffle=False)

# Models

In [ ]:
# This is the MLP used in the GP-MLP model
model = MLP(in_channels= len(predictors)-1, 
            hidden_channels=[50,50], 
            likelihood_fn='bgmm', # 'gamma', 'ggmm', bgmm', 'b2gmm', 'b2sgmm'
            dropout_rate=0,
           )

print(f'Number of parameters: {sum(p.numel() for p in model.parameters())}')

In [ ]:
# This is the MLP used as a baseline (on its own)
MLP = MLP(in_channels= len(predictors), 
            hidden_channels=[16,32,64,32,16], 
            likelihood_fn='bgmm', # 'gamma', 'ggmm', bgmm', 'b2gmm', 'b2sgmm'
            dropout_rate=0,
           )

print(f'Number of parameters: {sum(p.numel() for p in MLP.parameters())}')

In [ ]:
model.to(device)
MLP.to(device);

In [ ]:
def reset_weights(m):
    '''
    This function resets model weights randomly following the original initialization method.
    Works for simple architectures and commonly used layers.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            # If the layer has the method `reset_parameters`, it will be called
            layer.reset_parameters()

### MLP (baseline)

In [ ]:
MLP.apply(reset_weights);

In [ ]:
train_losses = []
val_losses = []
test_losses = []

train_loader = train_dataloader
valid_loader = val_dataloader
# test_loader = dataloader

epochs = 30

optimizer = torch.optim.Adam(MLP.parameters(), lr=0.0001)

for epoch in range(epochs):
    
    epoch_train_losses, epoch_val_losses = [], []
    
    MLP.train()
    
    for i, (inputs, labels) in enumerate(train_dataloader):
        
        inputs = inputs.float().to(device) # [batch_size, num_stations, num_predictiors]
        labels = labels.float().to(device) # [batch_size, num_stations]
        
        inputs = inputs.permute([0,2,1])
        batch_size = inputs.shape[0]
        
        optimizer.zero_grad()
        
        outputs = MLP(inputs)

        loss = loss_fn(outputs, labels, inputs, MLP, reduction='sum')/batch_size 
        # print('train: ', epoch, i, loss.item())
        
        if loss.item() != 0: 
            
            loss.backward()
            
            for param in optimizer.param_groups[0]['params']:
                nn.utils.clip_grad_value_(param, 1) # Bit of regularisation

            optimizer.step()
        
        epoch_train_losses.append(loss.item())
        
        #train_loss, val_loss, test_loss = train_epoch(MLP, optimizer, train_loader, valid_loader, test_loader=test_loader, print_progress=True, epoch=epoch, device=device, permute=True)

    model.eval()
    
    with torch.no_grad():
        
        for i, (inputs, labels) in enumerate(val_dataloader):

            inputs = inputs.float().to(device) # [batch_size, num_stations, num_predictiors]
            labels = labels.float().to(device) # [batch_size, num_stations]
        
            inputs = inputs.permute([0,2,1])
            batch_size = inputs.shape[0]
            
            outputs = MLP(inputs)
            
#             pdb.set_trace()
            
#             assert inputs.isnan().sum() == 0
#             assert outputs.isnan().sum() == 0
#             assert labels.isnan().sum() == 0

#             # device = labels.device
#             logp = torch.zeros_like(labels, device=device)
    
#             b_mask = labels == 0 # shape (b, n_samples, k)
#             g_mask = labels > 0

#             # shape of outputs: (b, num_stations, n_samples, num_output_dims)
   
#             pi = outputs[...,0] # shape (b, num_stations, n_samples)
#             alpha = outputs[...,1] # shape (b, num_stations, n_samples)
#             beta = outputs[...,2] # shape (b, num_stations, n_samples)

#             # Computing log probabilities for gamma distribution where obs > 0
#             gamma_dist = Gamma(concentration=alpha[g_mask], rate=beta[g_mask])
#             logp[g_mask] = torch.log((1 - pi[g_mask])) + gamma_dist.log_prob(labels[g_mask])
    
#             # Computing log probabilities for Bernoulli distribution where obs == 0
#             logp[b_mask] = torch.log(pi[b_mask])

#             if mask is not None:
#                 logp = logp * mask

            loss = loss_fn(outputs, labels, inputs, MLP, reduction='sum')/batch_size
            # print('val: ', epoch, i, loss.item())

            epoch_val_losses.append(loss.item())
    
    epoch_train_loss = np.mean(epoch_train_losses)/len(train_stations) 
    epoch_val_loss = np.mean(epoch_val_losses)/len(val_stations) #, np.mean(test_losses)
    
    print(f'epoch : {epoch+1}, train loss : {np.mean(epoch_train_loss):.4f} , valid loss : {np.mean(epoch_val_loss):.4f}')
   
    train_losses.append(epoch_train_loss)
    val_losses.append(epoch_val_loss)
#     test_losses.append(test_loss)

In [ ]:
plt.plot(np.array(train_losses), label='train')
plt.plot(np.array(val_losses), label='val')
plt.legend()
plt.show()

### GP-MLP

In [ ]:
model.apply(reset_weights);

In [ ]:
print_progress = True
plot_x_ind = True
validate_flag = True
f_marginal_flag = False
mc_samples = 5

In [ ]:
# arr, _ = UpperIndusGridDataset(root_folder, train_mean, train_var).__getitem__(10)
# x_min = arr[0,:].min()
# x_max = arr[0,:].max() 
# y_min = arr[1,:].min()
# y_max = arr[1,:].max()

In [ ]:
st = ds_dataset.st
st_val = ds_dataset_val.st
coords_train_stations = st.groupby('Station').mean()[['X','Y']].values
coords_val_stations = st_val.groupby('Station').mean()[['X','Y']].values

x_ind_stations = coords_train_stations

# coords_train_stations.shape, coords_val_stations.shape

In [ ]:
# Number of points for inducing points grid
# x_points = 30
# y_points = 30

#  Grid of equally spaced points covering the spatial extent for which station data is available
# grid = np.meshgrid(np.linspace(st['X'].min(),st['X'].max(),x_points),np.linspace(st['Y'].min(),st['Y'].max(),y_points))
# grid = np.meshgrid(np.linspace(x_min,x_max,x_points),np.linspace(y_min,y_max,y_points))

# Make grid into a tensor
# grid = torch.tensor(grid, dtype=torch.float32).flatten(start_dim=1).permute(1,0)

# x_ind = grid.clone().detach().requires_grad_(True)
# num_ind_points = x_points * y_points
    
# Inducing points are points in the station locations
x_ind = torch.tensor(x_ind_stations, dtype=torch.float32).detach().requires_grad_(False).to(device)
# x_ind = torch.nn.Parameter(torch.tensor(x_ind_stations, dtype=torch.float32, requires_grad=True).to(device))
num_ind_points = len(x_ind)

n = len(st)

# ls = torch.nn.Parameter(torch.tensor(2, dtype=torch.float32, requires_grad=True).to(device))

with Measure() as prior:
    #f1 = GP(EQ().stretch(100))
    #f2 = GP(EQ().stretch(10))
    #f3 = GP(EQ().stretch(ls))
    f3 = GP(Matern12)
    f = f3 #f1 + f2 + f3
 
q = ApproximatePosterior(num_ind_points) # q is the approximate posterior

optimizer = torch.optim.Adam(list(model.parameters())+list(q.parameters()), lr=10e-4)
# optimizer = torch.optim.Adam(list(model.parameters())+list(q.parameters())+[ls], lr=10e-4)

In [ ]:
load_model = False
exp_name = 'deep_MLP'

# Options for loading an existing model, otherwise start from scratch

if load_model:
    q.load_state_dict(torch.load(f'q_state_{exp_name}.pth'))
    optimizer.load_state_dict(torch.load(f'optimizer_state_{exp_name}.pth'))
    model.load_state_dict(torch.load(f'model_state_{exp_name}.pth'))

    # Load the data back
    with open(f'training_progress_{exp_name}.pkl', 'rb') as file:
        data_loaded = pickle.load(file)

    e_past = data_loaded["last_epoch"]
    train_loss = data_loaded["train_loss"]
    train_loglik = data_loaded["train_loglik"]
    train_kl = data_loaded["train_kl"]
    train_nll = data_loaded["train_nll"]
    val_loss = data_loaded["val_loss"]
    val_loglik = data_loaded["val_loglik"]
    val_kl = data_loaded["val_kl"]
    val_nll = data_loaded["val_nll"]

else:
    e_past = 0
    train_loss, train_loglik, train_kl, train_nll = [], [], [], []
    val_loss, val_loglik, val_kl, val_nll = [], [], [], []

test_loss, test_loglik, test_kl, test_nll  = [], [], [], []

train_loss_batch, train_loglik_batch, train_kl_batch, train_nll_batch = RunningAverage(), RunningAverage(), RunningAverage(), RunningAverage()
val_loss_batch, val_loglik_batch, val_kl_batch, val_nll_batch = RunningAverage(), RunningAverage(), RunningAverage(),RunningAverage()

In [ ]:
# plt.scatter(x_ind_stations[:,0],x_ind_stations[:,1],marker='o',label='old')
# new_x_ind = x_ind.detach().cpu().numpy()
# plt.scatter(new_x_ind[:,0],new_x_ind[:,1],marker='x', label='new')
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
num_epochs = 10
    
for e in range(e_past + 1, num_epochs):

    torch.cuda.empty_cache()
    # print_gpu_usage()
    
    # TRAIN EPOCH    
    model.train()

    start = time.time()

    train_loss_batch.reset()
    train_loglik_batch.reset()
    train_kl_batch.reset()
    train_nll_batch.reset()
    val_loss_batch.reset()
    val_loglik_batch.reset()
    val_kl_batch.reset()
    val_nll_batch.reset()

    n = train_dataloader.dataset.n
    
    for i, (inputs, labels) in enumerate(train_dataloader):

        inputs = inputs.to(device) # inputs [batch_size, num_predictors, num_stations]
        labels = labels.to(device) # labels [batch_size, num_stations]

        elbo, recon, kl, num_points, nll = forward_backward_pass(inputs, labels, n, model, optimizer, q, f, x_ind, 
                                                            inducing_points=True, backward=True, f_marginal=f_marginal_flag, n_samples=mc_samples)

        # Keep track of loss terms
        train_loss_batch.update(-elbo.item())
        train_loglik_batch.update(-recon.item())
        train_kl_batch.update(kl.item())
        train_nll_batch.update(nll.item()/num_points.item())

    elapsed = time.time() - start

    if print_progress:
        print(f'Train epoch  {e}: -elbo: {train_loss_batch.avg:.3f} | kl: {train_kl_batch.avg:.3f} | -recon: {train_loglik_batch.avg:.3f} | nll: {train_nll_batch.avg:.3f} -- time: {elapsed:.2f}')
        print(f'GP lengthscale: {ls.item()}')

    # Add average batch loss terms to lists
    train_loss.append(train_loss_batch.avg)
    train_loglik.append(train_loglik_batch.avg)
    train_kl.append(train_kl_batch.avg)
    train_nll.append(train_nll_batch.avg)
    
    # VALIDATION EPOCH
    if validate_flag:

        model.eval()

        n = val_dataloader.dataset.n
        
        start = time.time()

        with torch.no_grad():

            for i, (inputs, labels) in enumerate(val_dataloader):
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                elbo, recon, kl, num_points, nll = forward_backward_pass(inputs, labels, n, model, optimizer, q, f, x_ind, 
                                                                    inducing_points=True, backward=False, f_marginal=f_marginal_flag, n_samples=mc_samples*100)

            
                # Keep track of loss terms
                val_loss_batch.update(-elbo.item())
                val_loglik_batch.update(-recon.item())
                val_kl_batch.update(kl.item())
                val_nll_batch.update(nll.item()/num_points.item())

        elapsed_val = time.time() - start

        if print_progress:
            print(f'Val epoch {e}: -elbo: {val_loss_batch.avg:.3f} | kl: {val_kl_batch.avg:.3f} | -recon: {val_loglik_batch.avg:.3f} | nll: {val_nll_batch.avg:.3f} -- time: {elapsed_val:.2f}')

        # Add average batch loss terms to lists
        val_loss.append(val_loss_batch.avg)
        val_loglik.append(val_loglik_batch.avg)
        val_kl.append(val_kl_batch.avg)
        val_nll.append(val_nll_batch.avg)

In [ ]:
# Save the model and optimizer state

exp_name = 'deep_MLP'

torch.save(q.state_dict(), f'q_state_{exp_name}.pth')
torch.save(model.state_dict(), f'model_state_{exp_name}.pth')
torch.save(optimizer.state_dict(), f'optimizer_state_{exp_name}.pth')

data_to_save = {
    "last_epoch": e,
    "train_loss": train_loss,
    "train_loglik": train_loglik,
    "train_kl": train_kl,
    "train_nll": train_nll,
    "val_loss": val_loss,
    "val_loglik": val_loglik,
    "val_kl": val_kl,
    "val_nll": val_nll,
}

# Save the data to a file
with open(f'training_progress_{exp_name}.pkl', 'wb') as file:
    pickle.dump(data_to_save, file)

In [ ]:
train_vars = [train_loss, train_loglik, train_kl, train_nll] 
validation_vars = [val_loss, val_loglik, val_kl, val_nll]

labels = ['Negative ELBO','Negative log likelihood term','KL divergence term','NLL']

fig, axes = plt.subplots(4,1, figsize=(10,10))
for i,ax in enumerate(axes.flatten()):
    ax.plot(train_vars[i], label='train')
    ax.plot(validation_vars[i], label='val')
#     ax.set_title(f'{labels[i]} | best: {np.min(train_vars[i]):.2f}')
    ax.set_ylabel(f'{labels[i]}', fontsize=14)
    ax.set_xlabel('Epoch', fontsize=14)
    ax.set_xticks(np.arange(0, num_epochs))
#     ax.set_yscale('log')
plt.tight_layout()
plt.show()

### Test maps

In [ ]:
# test_dataloader = DataLoader(dataset=UpperIndusGridDataset(root_folder, train_mean, train_var), batch_size=1, shuffle=True)

inducing_points = True

model.eval()

# n = val_dataloader.dataset.n
        
start = time.time()

with torch.no_grad():

    for i, (inputs, aux_inputs) in enumerate(dataloader_map):
        
        inputs = inputs.permute(0,2,1).to(device)
        og_inputs = inputs.to('cpu')
#         pdb.set_trace()
        inputs, outputs = forward_backward_pass(inputs, None, n, model, optimizer, q, f, x_ind, 
                                                inducing_points=True, backward=False, f_marginal=f_marginal_flag, n_samples=1, test_time=True)

# model.eval()

# with torch.no_grad():
    
#     for index, (inputs, aux_inputs) in enumerate(dataloader_map):
        
#         inputs = inputs[:,:,:].to(device)
#         aux_inputs = aux_inputs[:,:,:].to(device)
        
#         b = inputs.shape[0]
        
#         if index == 0:
        
#             q.build_normal()

#             x = inputs[0,:2,:].permute(1,0).float()
            
#             inputs = inputs[:,2:,:]
#             aux_inputs = aux_inputs[:,:,:].to(device)

#             # Sample z and concatenate to inputs
#             if inducing_points:
                
#                 #f_sample = f_post(x).sample(b).permute(1,0).unsqueeze(1)
                
#                 f_post = f | (f(x_ind), q.sample())
                
#                 if f_marginal_flag:
#                     f_sample = Normal(f_post.mean(x),
#                                       Diagonal(f_post.kernel.elwise(x)[:, 0])
#                                       ).sample().permute(1,0).unsqueeze(1)
#                 else:
#                     f_sample = f_post(x).sample().permute(1,0).unsqueeze(1)
                
#                 inputs = torch.cat([f_sample, inputs], dim=1)
                 
#             else:
#                 q_sample = q.sample(b).permute(1,0).unsqueeze(1)
                
#                 inputs = torch.cat([q_sample, inputs], dim=1)

#             # Masking for missing data
#             inputs = inputs.permute(0,2,1)
#             aux_inputs = aux_inputs.permute(0,2,1)
 
#             mask = ~torch.any(inputs.isnan(),dim=2)
#             k = mask.sum()

#             # Forward pass
#             pdb.set_trace()
#             outputs = model(inputs[mask].float())

In [ ]:
# inputs = inputs.permute(0,2,1)
print(inputs.squeeze()[:,:].max(0).values)
print(inputs.squeeze()[:,:].min(0).values)

In [ ]:
col_names = ['X','Y','wrf_prcp',
#              'doy_sin','doy_cos',
             'GPsample','pi','alpha','beta']
#col_names = ['X','Y','Z','wrf_prcp','doy_sin','doy_cos','GPsample','alpha','beta']

data = np.concatenate([og_inputs.squeeze().permute(1,0).cpu().numpy()[:,:2],
                       inputs.squeeze().cpu().numpy()[:,9:10], #* np.sqrt(train_var[9:10]) + train_mean[9:10],
                       np.expand_dims(inputs.squeeze().cpu().numpy()[:,0], axis=1),
                       outputs.squeeze().cpu().numpy()], axis=1
                       )

# pdb.set_trace()

df = pd.DataFrame(data, columns=col_names)

gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.X, df.Y))

gdf['uniform'] = gdf.apply(lambda x: np.random.uniform(0,1),axis=1)
#gdf['uniform'] = np.random.uniform(0,1)

gdf['sample'] = gdf.apply(sample_apply, axis=1)

gdf['uniform'] = gdf.apply(lambda x: 0.5, axis=1)
gdf['bg_median'] = gdf.apply(sample_apply, axis=1)

gdf['g_mean'] = gdf['alpha']/gdf['beta']

gdf['p'] = 1 - gdf['pi']

x_ind_plot = x_ind.detach().cpu().numpy() * np.sqrt(train_var[:2]) + train_mean[:2] 
x_val_plot = val_stations * np.sqrt(train_var[:2]) + train_mean[:2] 

In [ ]:
st_stations = st.groupby('Station').mean()[['X','Y']] * np.sqrt(train_var[:2]) + train_mean[:2]

In [ ]:
gdf_stations = geopandas.GeoDataFrame(geometry=geopandas.points_from_xy(x_ind_plot[:,0], x_ind_plot[:,1]))
gdf_val_stations = geopandas.GeoDataFrame(geometry=geopandas.points_from_xy(x_val_plot[:,0], x_val_plot[:,1]))

plt.figure()
gdf_stations['q_sample'] = q.sample().detach().cpu().numpy()
plt.scatter(x_ind_plot[:,0],x_ind_plot[:,1], label='training locations')
plt.scatter(x_val_plot[:,0],x_val_plot[:,1], label='test locations')
# gdf_stations.plot('q_sample', cmap='viridis')
# gdf_val_stations.plot()
plt.gca().set_aspect('equal')
plt.legend()
plt.show()

# plt.plot(gdf_stations['geometry'].x, gdf_stations['q_sample'], 'o')

In [ ]:
variables = ['wrf_prcp',
             'p',
#              'g_mean',
#              'bg_median',
#              'Z',
             'sample',
             'GPsample'
            ]

labels = ['RCM simulated precip. (mm/day)', 
          'GP-MLP probability of precip. occurrence',
#           '(c) GP-MLP Gamma mean precip. (mm/day)',
#           '(d) GP-MLP median precip. (mm/day)',
#           'Elevation (m.a.s.l.)',
          'Sample from GP-MLP outputs (mm/day)', 
          'GP sample'
         ]

cmaps = ['terrain',
         'viridis',
         'terrain',
         'terrain',
#          'viridis',...
#          'terrain',
#          'viridis'
        ]

fig, axes = plt.subplots(2,2,figsize=(10,12))

for idx, ax in enumerate(axes.flatten()):
    
    if idx < len(variables):
    
    #     gdf.set_crs(catchments.crs)
    #     gdf_clip = geopandas.clip(gdf, catchments)
    
        gdf.plot(variables[idx], ax=ax, legend=True, cmap=cmaps[idx], legend_kwds={'shrink': 0.6})
#         beas.plot(ax=ax, facecolor='None', edgecolor='black')
        #gdf_stations.plot(ax=ax, color='red')
        #sutlej.plot(ax=ax, facecolor='None', edgecolor='black')

        ax.set_title(labels[idx],fontsize=15)
        
#         if variables[idx]=='GPsample':
#             ax.scatter(x_ind_plot[:,0],x_ind_plot[:,1], s=4, c='r')
#             ax.scatter(st_stations['X'],st_stations['Y'], s=3, c='w')

    ax.set_frame_on(False)
    ax.set_yticks([])
    ax.set_xticks([])     

plt.tight_layout()
# plt.savefig('maps2',dpi=300)